一、数据载入及初步观察

In [23]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import os
import time

In [24]:
#Read csv file
pf = pd.read_csv("train.csv", iterator=True, header = 0)
start = time.time()

In [25]:
chunksize = 1000
flag = True
chunks = []
while flag:
    try:
        chunk = pf.get_chunk(chunksize)
        chunks.append(chunk)
    except StopIteration:
        flag = False
        print("Iteration is stopped.")
df = pd.concat(chunks, ignore_index=True)
print("分块处理之后读取用时为：%0.2f s"%(time.time()-start))

Iteration is stopped.
分块处理之后读取用时为：0.44 s


In [26]:
#查看各列的数据类型
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [41]:
df.rename(columns={'PassengerId':'乘客ID','Survived':'是否幸存','Pclass':'乘客等级(1/2/3等舱位)','Name':'乘客姓名','Sex':'性别','Age':'年龄','SibSp':'堂兄弟/妹个数','Parch':'父母与小孩个数','Ticket':'船票信息','Fare':'票价','Cabin':'客舱','Embarked':'登船港口'},inplace=True)
df.head()

,乘客ID,是否幸存,乘客等级(1/2/3等舱位),乘客姓名,性别,年龄,堂兄弟/妹个数,父母与小孩个数,船票信息,票价,客舱,登船港口
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [42]:
#查看基本的数据信息
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   乘客ID            891 non-null    int64  
 1   是否幸存            891 non-null    int64  
 2   乘客等级(1/2/3等舱位)  891 non-null    int64  
 3   乘客姓名            891 non-null    object 
 4   性别              891 non-null    object 
 5   年龄              714 non-null    float64
 6   堂兄弟/妹个数         891 non-null    int64  
 7   父母与小孩个数         891 non-null    int64  
 8   船票信息            891 non-null    object 
 9   票价              891 non-null    float64
 10  客舱              204 non-null    object 
 11  登船港口            889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [43]:
#判断数据是否为空，为空的地方返回True，其余地方返回False
df.isnull().head()

,乘客ID,是否幸存,乘客等级(1/2/3等舱位),乘客姓名,性别,年龄,堂兄弟/妹个数,父母与小孩个数,船票信息,票价,客舱,登船港口
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False


In [44]:
#保存
ddf = df.to_csv("train_chinese.csv")

二、了解已加载的字段含义以及初步观察数据。这里基于pandas库来完成相关功能的实现。

Pandas 基于 Numpy开发，可以与其它第三方科学计算支持库完美集成。是 Python 核心数据分析支持库，提供了快速、灵活、明确的数据结构，旨在简单、直观地处理关系型、标记型数据。
Pandas中主要有两种该数据结构， Series（一维数据）与 DataFrame（二维数据）。

1.创建：

In [21]:
#用值列表或一维数组生成 Series时，Pandas 默认自动生成整数索引。
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

In [22]:
#除了这种方式，还能以字典对象直接生成：
average = {'Chinese': 120, 'Math': 131, 'English': 120, 'Chemistry': 79, 'Physics': 103}
averageList = pd.Series(average)
averageList

Chinese      120
Math         131
English      120
Chemistry     79
Physics      103
dtype: int64

In [21]:
#利用字典创建时也可以指定索引，索引和字典键相同才有效，否则默认为NaN
average = {'Chinese':120,'Math':131,'English':120,'Chemistry':79,'Physics':103}
averageList = pd.Series(average,index = ['Chinese', 'Math', 'English', 'Chemistry', 'Physics','history'])
averageList

Chinese      120.0
Math         131.0
English      120.0
Chemistry     79.0
Physics      103.0
history        NaN
dtype: float64

In [111]:
#对于DataFrame的生成方式就更多了，可以是用含日期时间索引与标签的 NumPy 数组生成，也可以用 Series 字典对象生成
#数组、列表或元组构成的字典构造dataframe
#构造一个字典
data = {'a':[1,2,3,4],'b':(5,6,7,8),'c':np.arange(9,13)}
frame = pd.DataFrame(data)
frame

,a,b,c
0,1,5,9
1,2,6,10
2,3,7,11
3,4,8,12


In [94]:
#ndarray 构造dataframe
#构造二维数组对象
arr1 = np.arange(12).reshape(4,3)
frame1 = pd.DataFrame(arr1)
frame1

,0,1,2
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11


In [96]:
# Series构成的列表构造dataframe
arr2 = [pd.Series(np.random.rand(3)),pd.Series(np.random.rand(2))]
frame2 = pd.DataFrame(arr2)
frame2

,0,1,2
0,0.032339,0.091441,0.739935
1,0.280405,0.515588,NaN


2.常用属性和方法：

Series的常用属性:
shape 返回形状
 size 返回元素个数
 index 返回索引值
 values 返回值
 dtype 元素的类型
 columns 返回列名

In [26]:
averageList.dtype

dtype('float64')

In [27]:
averageList.size

6

In [29]:
averageList.shape

(6,)

Series的常用方法：
    head(n),tail(n) 读取前/后n行数据
    unique() 去重
    isnull(),notnull() isnull()用于判断每一个元素是否为空，为空返回True，否则返回False，notnull()与之相反
    add() sub() mul() div()  加减乘除的运算

In [34]:
averageList.head(2)

Chinese    120.0
Math       131.0
dtype: float64

DataFrame的常用属性: shape 返回形状 size 返回元素个数 index 返回索引值 values 返回值 dtype 元素的类型 columns 返回列名

In [98]:
frame.index

RangeIndex(start=0, stop=4, step=1)

In [99]:
frame.columns

Index(['a', 'b', 'c'], dtype='object')

In [112]:
#添加和删除列的方法
frame['d'] = [13, 14, 15, 16]
frame

,a,b,c,d
0,1,5,9,13
1,2,6,10,14
2,3,7,11,15
3,4,8,12,16


In [115]:
#T转置（行列变化）
frame.T

,0,1,2,3
a,1,2,3,4
b,5,6,7,8
c,9,10,11,12
d,13,14,15,16


In [116]:
del(frame['a'])
frame

,b,c,d
0,5,9,13
1,6,10,14
2,7,11,15
3,8,12,16


3.筛选操作

我们通常使用优化过的 Pandas 数据访问方法：.at、.iat、.loc 和 .iloc

In [86]:
dftr = pd.read_csv('train.csv')
dftr.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [71]:
#直接查找某一列的数据
dftr.Cabin.head()

0     NaN
1     C85
2     NaN
3    C123
4     NaN
Name: Cabin, dtype: object

In [62]:
#用标签选择需要的多列数据
dftr.loc[:, ['Survived', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Cabin', 'Embarked']]

,Survived,Pclass,Sex,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,1,0,7.2500,NaN,S
1,1,1,female,1,0,71.2833,C85,C
2,1,3,female,0,0,7.9250,NaN,S
3,1,1,female,1,0,53.1000,C123,S
4,0,3,male,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
886,0,2,male,0,0,13.0000,NaN,S
887,1,1,female,0,0,30.0000,B42,S
888,0,3,female,1,2,23.4500,NaN,S
889,1,1,male,0,0,30.0000,C148,C


In [68]:
#切片出第一行数据
dftr.iloc[0]

PassengerId                          1
Survived                             0
Pclass                               3
Name           Braund, Mr. Owen Harris
Sex                               male
Age                               22.0
SibSp                                1
Parch                                0
Ticket                       A/5 21171
Fare                              7.25
Cabin                              NaN
Embarked                             S
Name: 0, dtype: object

In [84]:
#切片出二十五到三十五行数据
dftr.loc[25:35]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
25,26,1,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,347077,31.3875,NaN,S
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,C23 C25 C27,S
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q
29,30,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,349216,7.8958,NaN,S
30,31,0,1,"Uruchurtu, Don. Manuel E",male,40.0,0,0,PC 17601,27.7208,NaN,C
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,1,0,PC 17569,146.5208,B78,C
32,33,1,3,"Glynn, Miss. Mary Agatha",female,NaN,0,0,335677,7.7500,NaN,Q
33,34,0,2,"Wheadon, Mr. Edward H",male,66.0,0,0,C.A. 24579,10.5000,NaN,S
34,35,0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,NaN,C


In [79]:
#按某列的某一值进行查找
dftr[dftr['Sex'].isin(['female'])]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...,...
880,881,1,2,"Shelley, Mrs. William (Imanita Parrish Hall)",female,25.0,0,1,230433,26.0000,NaN,S
882,883,0,3,"Dahlberg, Miss. Gerda Ulrika",female,22.0,0,0,7552,10.5167,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


In [76]:
#针对某一列的值进行修改
dftr[dftr['Age']<10].head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
16,17,0,3,"Rice, Master. Eugene",male,2.0,4,1,382652,29.1250,NaN,Q
24,25,0,3,"Palsson, Miss. Torborg Danira",female,8.0,3,1,349909,21.0750,NaN,S
43,44,1,2,"Laroche, Miss. Simonne Marie Anne Andree",female,3.0,1,2,SC/Paris 2123,41.5792,NaN,C


In [77]:
midage = df[(df["Age"]>10)& (df["Age"]<50)]
midage.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


reset_index（level=None， drop=False， name=NoDefault.no_default， inplace=False)
生成新的数据帧或序列，并重置索引。当需要将索引视为列时，或者当索引无意义且需要在另一个操作之前重置为默认值时，这很有用。

In [90]:
#midage = midage.reset_index(drop=True)
midage.loc[[100,105,108],['Pclass','Name','Sex']] 

,Pclass,Name,Sex
100,2,"Byles, Rev. Thomas Roussel Davids",male
105,3,"Cribb, Mr. John Hatfield",male
108,3,"Calic, Mr. Jovo",male


In [91]:
midage.iloc[[100,105,108],[2,3,4]]

,Pclass,Name,Sex
100,2,"Byles, Rev. Thomas Roussel Davids",male
105,3,"Cribb, Mr. John Hatfield",male
108,3,"Calic, Mr. Jovo",male


.loc和.iloc的差异：

相同点：（1）操作对象相同：loc和iloc都是对DataFrame类型进行操作；（2）完成目的相同：二者都是用于选取DataFrame中对应行或列中的元素。

不同点：（1）loc和iloc索引的行列标签类型不同；（2）iloc使用顺序数字来索引数据，而不能使用字符型的标签来索引数据（这里的顺序数字是指从0开始计数！）；（3）loc使用实际设置的索引来索引数据。但行列名为数字时，loc也可以索引数字，但这里的数字不一定从0开始编号，是对应具体行列名的数字！

4.计算操作

在进行计算时，有两个要点需要注意：1.运算时排除缺失值。2.不同维度对象运算时，要先对齐。（此外，Pandas 自动沿指定维度广播,扩展出的维度由于原有值的缺失，全部标记为NaN。）

In [117]:
frame1_a = pd.DataFrame(np.arange(9.).reshape(3, 3),columns=['a', 'b', 'c'],index=['one', 'two', 'three'])
frame1_b = pd.DataFrame(np.arange(12.).reshape(4, 3),columns=['a', 'b', 'c'],index=['first', 'one', 'two', 'second'])
frame1_a

,a,b,c
one,0.0,1.0,2.0
two,3.0,4.0,5.0
three,6.0,7.0,8.0


In [118]:
frame1_b

,a,b,c
first,0.0,1.0,2.0
one,3.0,4.0,5.0
two,6.0,7.0,8.0
second,9.0,10.0,11.0


In [120]:
frame1_sum = frame1_a + frame1_b
frame1_sum

,a,b,c
first,NaN,NaN,NaN
one,3.0,5.0,7.0
second,NaN,NaN,NaN
three,NaN,NaN,NaN
two,9.0,11.0,13.0


In [125]:
frame1_c = frame1_b.rename(index = {'second':'three'})
frame1_div = frame1_a / frame1_c
frame1_div

,a,b,c
first,NaN,NaN,NaN
one,0.000000,0.250000,0.400000
three,0.666667,0.700000,0.727273
two,0.500000,0.571429,0.625000


三、探索性数据分析

In [75]:
s = np.arange(32).reshape((4, 8))
test = pd.DataFrame(s,index = ['one', 'two', 'three', 'four'], columns = ['d', 'a', 'b', 'c', 'e', 'h', 'f', 'g'] )

In [76]:
#对行或列索引进行排序（按字典顺序），可使用sort_index方法，它将返回一个已排序的新对象
test.sort_index()

,d,a,b,c,e,h,f,g
four,24,25,26,27,28,29,30,31
one,0,1,2,3,4,5,6,7
three,16,17,18,19,20,21,22,23
two,8,9,10,11,12,13,14,15


In [81]:
test.sort_values(by='b')
#同时对

,d,a,b,c,e,h,f,g
one,0,1,2,3,4,5,6,7
two,8,9,10,11,12,13,14,15
three,16,17,18,19,20,21,22,23
four,24,25,26,27,28,29,30,31


In [83]:
#按照列索引进行降序排序
test.sort_index(axis=1, ascending=False)

,h,g,f,e,d,c,b,a
one,5,7,6,4,0,3,2,1
two,13,15,14,12,8,11,10,9
three,21,23,22,20,16,19,18,17
four,29,31,30,28,24,27,26,25


In [80]:
pf3 = pd.read_csv("train_chinese.csv")

In [46]:
pf3.columns

Index(['Unnamed: 0', '乘客ID', '是否幸存', '乘客等级(1/2/3等舱位)', '乘客姓名', '性别', '年龄',
       '堂兄弟/妹个数', '父母与小孩个数', '船票信息', '票价', '客舱', '登船港口'],
      dtype='object')

In [47]:
pf3.sort_index()

,Unnamed: 0,乘客ID,是否幸存,乘客等级(1/2/3等舱位),乘客姓名,性别,年龄,堂兄弟/妹个数,父母与小孩个数,船票信息,票价,客舱,登船港口
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [131]:
#按照年龄和票价两列综合进行降序排序，在排序时，任何缺失值默认都会被放到Series的末尾
pf3.sort_values(by = ['票价', '年龄'], ascending=False).head()

,Unnamed: 0,乘客ID,是否幸存,乘客等级(1/2/3等舱位),乘客姓名,性别,年龄,堂兄弟/妹个数,父母与小孩个数,船票信息,票价,客舱,登船港口
679,679,680,1,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C
258,258,259,1,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,NaN,C
737,737,738,1,1,"Lesurer, Mr. Gustave J",male,35.0,0,0,PC 17755,512.3292,B101,C
438,438,439,0,1,"Fortune, Mr. Mark",male,64.0,1,4,19950,263.0000,C23 C25 C27,S
341,341,342,1,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.0000,C23 C25 C27,S


In [90]:
pf3.sort_values(by = ['票价', '年龄'], ascending=False).tail()

,Unnamed: 0,乘客ID,是否幸存,乘客等级(1/2/3等舱位),乘客姓名,性别,年龄,堂兄弟/妹个数,父母与小孩个数,船票信息,票价,客舱,登船港口
481,481,482,0,2,"Frost, Mr. Anthony Wood ""Archie""",male,NaN,0,0,239854,0.0,NaN,S
633,633,634,0,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.0,NaN,S
674,674,675,0,2,"Watson, Mr. Ennis Hastings",male,NaN,0,0,239856,0.0,NaN,S
732,732,733,0,2,"Knight, Mr. Robert J",male,NaN,0,0,239855,0.0,NaN,S
815,815,816,0,1,"Fry, Mr. Richard",male,NaN,0,0,112058,0.0,B102,S


In [128]:
biggestfamily = max(pf3['堂兄弟/妹个数'] + pf3['父母与小孩个数'])
biggestfamily

10

In [129]:
pf3['票价'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: 票价, dtype: float64

In [132]:
pf3['父母与小孩个数'].describe()

count    891.000000
mean       0.381594
std        0.806057
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        6.000000
Name: 父母与小孩个数, dtype: float64